In [1]:
import torch
torch.set_printoptions(sci_mode=False)
from torch.func import grad, vjp, jacrev
from model_triton import *

aa = torch.randn((2,3)) #torch.randn((2, 2, 3))
print(aa)
print("----")

# RELU
# print(t_relu_bkwd(aa) * aa)
# print(vjp(t_relu_fwd, aa)[0])
#print(jacrev(t_relu_fwd)(aa))

# GELU
# print(jacrev(t_gelu_fwd)(aa))
# print(t_gelu_bkwd(aa))
#print(t_gelu_bkwd(aa) * aa)

# LOG_SOFTMAX
# print(jacrev(t_log_softmax_fwd)(aa))
# print(t_log_softmax_bkwd(aa))

# Embed
# from model_torch_func import init_proj_layer
# params = (init_proj_layer(4, 2), )
# aa =torch.tensor([1, 0, 1])
# print(jacrev(t_embed_fwd)(params, aa))
# print(t_embed_bkwd(params, aa))

# Proj
# aa = torch.randn((2,2))
# #print(aa)
# params = torch.randn((4,2))
# #print(t_proj_fwd(params, aa))
# res = jacrev(t_proj_fwd)(params, aa)
# print(res.shape)
# print(res)
# res2 = t_proj_bkwd_p(params, aa)
# print(res2.shape)
# print(res2)
# res = jacrev(t_proj_fwd, argnums=1)(params, aa)
# print(res.shape, res)
# res2 = t_proj_bkwd_x(params, aa)
# print(res2.shape, res2)

# Linear
# aa = torch.randn((2,2))
# print(aa)
# params = (torch.randn((4,2)), torch.randn((4,)))
# res = jacrev(t_linear_fwd)(params, aa)
# print(res[0].shape, res[1].shape)
# print(res)
# res2 = t_linear_bkwd_p(params, aa)
# print(res2[0].shape, res2[1].shape)
# print(res2)

# from model_triton import t_linear_bkwd_x
# res = jacrev(t_linear_fwd, argnums=1)(params, aa)
# print(res.shape, res)
# res2 = t_linear_bkwd_x(params, aa)
# print(res2.shape, res2)

# LAYERNORM
# x = torch.randn((2,3), device="cuda")
# print(x)
# print(torch.mean(x, axis=-1, keepdims=True), torch.std(x, axis=-1, keepdims = True))
# from model_torch_func import init_layernorm_layer
# #params = init_layernorm_layer(3) # replace 1, 0s
# params = torch.randn((3, ), device="cuda"), torch.zeros((3,), device="cuda")

# from model_triton import t_layernorm_bkwd_p
# res1 = jacrev(t_layernorm_fwd)(params, x)
# res2 = t_layernorm_bkwd_p(params, x)
# print(res1[0].shape, res1[0])
# print("--")
# print(res2[0].shape, res2[0])

# from model_triton import t_layernorm_bkwd_x
# res1 = jacrev(t_layernorm_fwd, argnums=1)(params, x)
# res2 = t_layernorm_bkwd_x(params, x)
# print(res1.shape, res1)
# print("--")
# print(res2.shape, res2)


# TLAYER_FFN
# aa = torch.randn((3,2), device="cuda")
# print(aa)
# from model_torch_func import init_tlayer_ffn
# from model_triton import t_tlayer_ffn_fwd, t_gelu_fwd, t_linear_bkwd_p, t_gelu_bkwd
# from functools import partial
# fn = partial(t_tlayer_ffn_fwd, activation_fn=t_gelu_fwd)
# params = init_tlayer_ffn(2, 4)
# print(fn(params, aa))
# print(params[0].shape, params[1].shape)
# print("--")

# res = jacrev(fn, argnums=0)(params, aa)
# print(res[0].shape, res[1].shape, res[2].shape, res[3].shape)
# print(res[0],res[1])
# #print(res[2], res[3])

# res2 = t_tlayer_ffn_bkwd_p(params, aa, t_gelu_fwd)
# print(res2[0].shape, res2[1].shape, res2[2].shape, res2[3].shape)
# print(res2[0], res2[1])
# print(res2[2], res2[3])

# res = jacrev(fn, argnums=1)(params, aa)
# print(res.shape)
# print(res)

# res2 = t_tlayer_ffn_bkwd_x(params, aa, t_gelu_fwd)
# print(res2.shape)
# print(res2)

tensor([[-1.9953, -1.1435,  0.5128],
        [ 0.3337,  0.3123, -1.2417]])
----


In [4]:
# Embed
from model_torch_func import init_proj_layer
params = (init_proj_layer(4, 2), )
#aa =torch.tensor([1, 0, 1])
aa = torch.randint(2, (2, 6))
res=jacrev(t_embed_fwd)(params, aa)
print(len(res), res[0].shape)
print(res[0])

res2=t_embed_bkwd(params, aa)
print(len(res2), res2[0].shape)
print(res2[0])

1 torch.Size([2, 6, 4, 2, 4])
tensor([[[[[0., 0., 0., 0.],
           [2., 0., 0., 0.]],

          [[0., 0., 0., 0.],
           [0., 2., 0., 0.]],

          [[0., 0., 0., 0.],
           [0., 0., 2., 0.]],

          [[0., 0., 0., 0.],
           [0., 0., 0., 2.]]],


         [[[0., 0., 0., 0.],
           [2., 0., 0., 0.]],

          [[0., 0., 0., 0.],
           [0., 2., 0., 0.]],

          [[0., 0., 0., 0.],
           [0., 0., 2., 0.]],

          [[0., 0., 0., 0.],
           [0., 0., 0., 2.]]],


         [[[2., 0., 0., 0.],
           [0., 0., 0., 0.]],

          [[0., 2., 0., 0.],
           [0., 0., 0., 0.]],

          [[0., 0., 2., 0.],
           [0., 0., 0., 0.]],

          [[0., 0., 0., 2.],
           [0., 0., 0., 0.]]],


         [[[2., 0., 0., 0.],
           [0., 0., 0., 0.]],

          [[0., 2., 0., 0.],
           [0., 0., 0., 0.]],

          [[0., 0., 2., 0.],
           [0., 0., 0., 0.]],

          [[0., 0., 0., 2.],
           [0., 0., 0., 0.]]],


  

In [2]:
# TLAYER_FFN
aa = torch.randn((1, 3,2), device="cuda")
print(aa)
from model_torch_func import init_tlayer_ffn
from model_triton import t_tlayer_ffn_fwd, t_gelu_fwd, t_linear_bkwd_p, t_gelu_bkwd
from functools import partial
fn = partial(t_tlayer_ffn_fwd, activation_fn=t_gelu_fwd)
params = init_tlayer_ffn(2, 4)
print(fn(params, aa))
print(params[0].shape, params[1].shape)
print("--")

# res = jacrev(fn, argnums=0)(params, aa)
# print(res[0].shape, res[1].shape, res[2].shape, res[3].shape)
# print(res[0],res[1])
# print(res[2], res[3])
# print("--")

# res2 = t_tlayer_ffn_bkwd_p(params, aa, t_gelu_fwd)
# print(res2[0].shape, res2[1].shape, res2[2].shape, res2[3].shape)
# print(res2[0], res2[1])
# print(res2[2], res2[3])

res = jacrev(fn, argnums=1)(params, aa)
print(res.shape)
print(res)
print("--")

# def t_tlayer_ffn_bkwd_x(layer_params, x, activation_fn):
#     x_2d = x.reshape((-1, x.shape[-1]))
    
#     act_fn_bkwd = t_gelu_bkwd if activation_fn==t_gelu_fwd else t_relu_bkwd
    
#     dffn1_dx = t_linear_bkwd_x((layer_params[0], layer_params[1]), x_2d)
#     x_2d = t_linear_fwd((layer_params[0], layer_params[1]), x_2d)
#     dact_dx = act_fn_bkwd(x_2d)
#     x_2d = activation_fn(x_2d)
#     dffn2_dx = t_linear_bkwd_x((layer_params[2], layer_params[3]), x_2d)
#     dffn2_act_dx = dact_dx * dffn2_dx #Note dact_dx is only 2D, but torch will add other dims
#     jac = torch.einsum('abcd,cdef->abef', dffn2_act_dx, dffn1_dx)
#     return jac.reshape(x.shape+x.shape)

res2 = t_tlayer_ffn_bkwd_x(params, aa, t_gelu_fwd)
print(res2.shape)
print(res2)

tensor([[[-0.9504,  0.5064],
         [-1.1646, -0.3184],
         [ 0.3456,  0.1814]]], device='cuda:0')
tensor([[[    -0.0001,     -0.0001],
         [     0.0002,      0.0002],
         [    -0.0001,     -0.0001]]], device='cuda:0')
torch.Size([4, 2]) torch.Size([4])
--
torch.Size([1, 3, 2, 1, 3, 2])
tensor([[[[[[    -0.0001,     -0.0004],
            [     0.0000,      0.0000],
            [     0.0000,      0.0000]]],


          [[[    -0.0001,     -0.0003],
            [     0.0000,      0.0000],
            [     0.0000,      0.0000]]]],



         [[[[     0.0000,      0.0000],
            [    -0.0001,     -0.0004],
            [     0.0000,      0.0000]]],


          [[[     0.0000,      0.0000],
            [    -0.0001,     -0.0003],
            [     0.0000,      0.0000]]]],



         [[[[     0.0000,      0.0000],
            [     0.0000,      0.0000],
            [    -0.0001,     -0.0004]]],


          [[[     0.0000,      0.0000],
            [     0.0000,      

/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:767: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at /opt/pytorch/pytorch/aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


In [2]:
import torch
from model_triton import *
aa = torch.arange(8).reshape((2,2,2))
params = torch.arange(12).reshape((2, 3, 2))

print(torch.matmul(aa, params.reshape((-1, params.shape[-1])).transpose(-2, -1)).)
print(torch.matmul(aa, params.transpose(-2, -1)))

SyntaxError: invalid syntax (454213176.py, line 6)

In [11]:
params.transpose(-2, -1)

tensor([[[ 0,  1],
         [ 2,  3],
         [ 4,  5]],

        [[ 6,  7],
         [ 8,  9],
         [10, 11]],

        [[12, 13],
         [14, 15],
         [16, 17]]])

In [24]:
# Proj
import torch
torch.set_printoptions(sci_mode=False)
from torch.func import grad, vjp, jacrev
from model_triton import *
aa = torch.randn((1, 3, 2, 2))
#print(aa)
params = torch.randn((2, 3, 3, 2))
output = t_proj_fwd(params, aa)
#print(output.shape, output)
print("--")
res = jacrev(t_proj_fwd)(params, aa)
print(res.shape)
#print(res)
print("--")
res2 = t_proj_bkwd_p(params, aa)
print(res2.shape)
#print(res2)
# res = jacrev(t_proj_fwd, argnums=1)(params, aa)
# print(res.shape, res)
# res2 = t_proj_bkwd_x(params, aa)
# print(res2.shape, res2)

--
torch.Size([2, 3, 2, 3, 2, 3, 3, 2])
--


RuntimeError: shape '[1, 3, 2, 3, 2, 3, 3, 2]' is invalid for input of size 108

In [5]:
# t_scaled_dot_prod_attn
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_triton import *
BS, H, N, D = 1, 1, 2, 3
qkv= torch.randn((BS, H, 3, N, D), device="cuda")
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
from functools import partial

def attn_fwd(q, k, mask):
    attn = torch.matmul(q, torch.transpose(k, -2, -1))
    attn = attn / math.sqrt(D)
    attn = torch.where(torch.unsqueeze(mask,dim=1), attn, torch.full_like(attn, -1e9)) # Note, instead of usign -jnp.inf, which results in NaNs (NIT: probably better to use jax.numpy.finfo)
    return torch.exp(t_log_softmax_fwd(attn)) #TODO XXX: numerical stability

# q, k, _ = torch.unbind(qkv, dim=2)
# #fn = partial(attn_fwd, mask=mask) #this works
# #fn = partial(softmax_attn, mask=mask, train=False)
# res1 = jacrev(fn)(q, k)
# res2 = jacrev(fn, argnums=1)(q, k)
# print(res1.shape, res2.shape)
# print(res1)
# print(res2)
# print("--")

fn = partial(t_scaled_dot_prod_attn_fwd, mask=mask, train=False)
res = jacrev(fn)(qkv)
# print(res.shape)
dq,dk,dv = res.unbind(-3)
print(dq.shape)
print(dq)
#print(dk)
#print(dv)
print("--")

### Repro for numerical isntability!
def attn_bkwd(q, k, mask): # Do all ops in 3d instead of 4d
    attn = torch.matmul(q, torch.transpose(k, -2, -1))
    attn = attn / math.sqrt(D)
    attn = torch.where(torch.unsqueeze(mask,dim=1), attn, torch.full_like(attn, -1e9)) # Note, instead of usign -jnp.inf, which results in NaNs (NIT: probably better to use jax.numpy.finfo)
    dattn_dx = torch.exp(t_log_softmax_fwd(attn))[..., None, None, None, None] * t_log_softmax_bkwd(attn) 
    #dattn_dx = t_log_softmax_bkwd(attn) # TODO XXX: numerical stability
    jac1 = torch.matmul(dattn_dx, k/math.sqrt(D))
    jac2 = torch.matmul(q.transpose(-2,-1)/math.sqrt(D), dattn_dx).transpose(-2,-1)
    return jac1, jac2

# This works modulo numerical instability
# res2 = attn_bkwd(q, k, mask)
# print(res2[0].shape, res2[1].shape)
# print(res2[0])
# print(res2[1])

# This works modulo numerical instability
# res2 = t_softmax_attn_bkwd(q, k, mask, False)
# print(res2[0].shape, res2[1].shape)
# print(res2[0])
# print(res2[1])

res2 = t_scaled_dot_prod_attn_bkwd(qkv, mask, False)
print(res2[0].shape) #, res2[1].shape, res2[2].shape)
print(res2[0])
#print(res2[1])
#print(res2[2])

torch.Size([1, 1, 2, 3, 1, 1, 2, 3])
tensor([[[[[[[[0.179492, 0.046042, 0.011952],
              [0.000000, 0.000000, 0.000000]]]],



           [[[[0.632378, 0.162211, 0.042108],
              [0.000000, 0.000000, 0.000000]]]],



           [[[[0.905338, 0.232228, 0.060283],
              [0.000000, 0.000000, 0.000000]]]]],




          [[[[[0.000000, 0.000000, 0.000000],
              [0.094128, 0.024145, 0.006268]]]],



           [[[[0.000000, 0.000000, 0.000000],
              [0.331628, 0.085066, 0.022082]]]],



           [[[[0.000000, 0.000000, 0.000000],
              [0.474773, 0.121784, 0.031613]]]]]]]], device='cuda:0')
--
torch.Size([1, 1, 2, 3, 1, 1, 2, 3])
tensor([[[[[[[[0.179492, 0.046042, 0.011952],
              [0.000000, 0.000000, 0.000000]]]],



           [[[[0.000000, 0.000000, 0.000000],
              [0.094128, 0.024145, 0.006268]]]],



           [[[[0.632378, 0.162211, 0.042108],
              [0.000000, 0.000000, 0.000000]]]]],




          [[[[[0.00

/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:767: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at /opt/pytorch/pytorch/aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


In [2]:
# t_tlayer_attn_heads_fwd
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_tlayer_attn_heads
from model_triton import *
BS, H, N, D = 1, 2, 2, 4
layer_params = init_tlayer_attn_heads(D, H)
qkv= torch.randn((BS, 3, N, D), device="cuda").unbind(-3)
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
from functools import partial

# fn = partial(t_tlayer_attn_heads_fwd, mask=mask, train=False)
# res = jacrev(fn)(layer_params, qkv)
# print(res.shape)
# #print(res)
# print("--")
# res2 = t_tlayer_attn_heads_bkwd_p(layer_params, qkv, mask, False)
# print(res2.shape)

fn = partial(t_tlayer_attn_heads_fwd, mask=mask, train=False)
res = jacrev(fn, argnums=1)(layer_params, qkv)
print(res[0].shape)
#print(res)
print("--")
res2 = t_tlayer_attn_heads_bkwd_x(layer_params, qkv, mask, False)
print(res2[0].shape)

torch.Size([1, 2, 2, 2, 1, 2, 4])
--
torch.Size([1, 2, 2, 2, 1, 2, 4])


In [2]:
# t_tlayer_attn_fwd
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_tlayer_attn
from model_triton import *
BS, H, N, D = 1, 1, 2, 4
layer_params = init_tlayer_attn(D, H)
qkv= torch.randn((BS, 3, N, D), device="cuda").unbind(-3)
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
from functools import partial
# fn = partial(t_tlayer_attn_fwd, mask=mask, train=False)
# # res = jacrev(fn)(layer_params, qkv)
# # print(len(res), res[0].shape, res[1].shape)
# # print(res[0][0][0][0])
# # #print(res)
# # print("--")
# # res2 = t_tlayer_attn_bkwd_p(layer_params, qkv, mask, False)
# # print(len(res2), res2[0].shape, res2[1].shape)
# # print(res2[0][0][0][0])

# ####

# res = jacrev(fn, argnums=1)(layer_params, qkv)
# print(len(res), res[0].shape, res[1].shape)
# print(res[0][0][0][0])

# res2 = t_tlayer_attn_bkwd_x(layer_params, qkv, mask, False)
# print(len(res2), res2[0].shape, res2[1].shape)
# print(res2[0][0][0][0])

In [1]:
# t_gpt2_tlayer_fwd_sublock1_fwd
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_tlayer_gpt2
from model_triton import *
BS, H, N, D = 1, 1, 2, 4
layer_params = init_tlayer_gpt2(D, H, 4*D, 1)
y= torch.randn((BS, N, D), device="cuda")
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
res = t_gpt2_tlayer_sublock1_fwd(layer_params[:-6], y, mask, False)
print(res.shape)
print("---")

from functools import partial
fn = partial(t_gpt2_tlayer_sublock1_fwd, mask=mask, train=False)
# res = jacrev(fn)(layer_params[:-6], y)
# print(len(res), res[0].shape, res[1].shape, res[2].shape, res[3].shape)
# print(res[1])
# #print(res[1])
# print("---")

# res2 = t_gpt2_tlayer_sublock1_bkwd_p(layer_params[:-6], y, mask, train=False)
# print(len(res2), res2[0].shape, res2[1].shape, res2[2].shape, res2[3].shape)
# print(res2[1])

res = jacrev(fn, argnums=1)(layer_params[:-6], y)
print(res.shape, res)
print("---")

# def t_gpt2_tlayer_sublock1_bkwd_x(layer_params, y, mask, train=True): # input: seq_len x emb_dim
#     y_diff = t_layernorm_fwd(layer_params[:2], y)
#     jac_layernorm_x = t_layernorm_bkwd_x(layer_params[:2], y)
#     y = y + t_dropout(t_tlayer_attn_fwd(layer_params[2:], (y_diff, y_diff, y_diff), mask, train), train)
#     # TODO XXX: add dropout!
#     jac_tlayer_attn_x = t_tlayer_attn_bkwd_x(layer_params[2:], (y_diff, y_diff, y_diff), mask, train)
    
#     jac_y = torch.eye(y.numel(), device=y.device)    
#     jac_tlayer_attn_x = torch.stack(jac_tlayer_attn_x)
#     jac_y_diff = torch.einsum("xabcdef, defghi->abcghi", jac_tlayer_attn_x, jac_layernorm_x)
#     return jac_y.reshape(jac_y_diff.shape)[None, ] + jac_y_diff

res2 = t_gpt2_tlayer_sublock1_bkwd_x(layer_params[:-6], y, mask, train=False)
print(res2.shape, res2)

torch.Size([1, 2, 4])
---
torch.Size([1, 2, 4, 1, 2, 4]) tensor([[[[[[ 9.998097e-01, -2.875166e-05,  1.540709e-04,  6.500125e-05],
            [-4.817256e-04, -1.268953e-04,  5.714922e-04,  3.712886e-05]]],


          [[[ 2.011918e-04,  1.000168e+00, -1.218463e-04, -2.473996e-04],
            [ 6.724527e-04,  2.545502e-04, -3.880580e-04, -5.389448e-04]]],


          [[[ 7.207511e-05,  1.075177e-05,  9.999416e-01, -2.443895e-05],
            [ 1.804511e-04,  8.391979e-05, -2.151169e-05, -2.428592e-04]]],


          [[[-9.842351e-05, -3.943307e-05,  7.235640e-05,  1.000065e+00],
            [-2.822778e-04, -7.325434e-06,  6.896323e-04, -4.000292e-04]]]],



         [[[[-1.903173e-04, -2.874095e-05,  1.540713e-04,  6.498698e-05],
            [ 9.995183e-01, -1.269059e-04,  5.714889e-04,  3.716303e-05]]],


          [[[ 2.011959e-04,  1.680674e-04, -1.218459e-04, -2.474174e-04],
            [ 6.724274e-04,  1.000255e+00, -3.880622e-04, -5.389022e-04]]],


          [[[ 7.207593e-05,  

In [3]:
# t_gpt2_tlayer_fwd_sublock2_fwd
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_tlayer_gpt2
from model_triton import *
BS, H, N, D = 2, 1, 2, 4
layer_params = init_tlayer_gpt2(D, H, 4*D, 1)
y= torch.randn((BS, N, D), device="cuda")
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
y = t_gpt2_tlayer_sublock1_fwd(layer_params[:-6], y, mask, False)
res = t_gpt2_tlayer_sublock2_fwd(layer_params[-6:], y, False)
print(res.shape)
print("---")

from functools import partial
fn = partial(t_gpt2_tlayer_sublock2_fwd, train=False)
# res = jacrev(fn)(layer_params[-6:], y)
# print(len(res), res[0].shape, res[1].shape, res[2].shape, res[3].shape)
# print(res[1])
# print("---")

res = jacrev(fn, argnums=1)(layer_params[-6:], y)
print(res.shape, res)
print("---")

res2 = t_gpt2_tlayer_sublock2_bkwd_x(layer_params[-6:], y, train=False)
print(res2.shape, res2)

torch.Size([2, 2, 4])
---
torch.Size([2, 2, 4, 2, 2, 4]) tensor([[[[[[ 1.000018e+00,  1.035568e-04, -6.908900e-06, -1.145899e-04],
            [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00]],

           [[ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00],
            [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00]]],


          [[[ 4.272713e-05,  1.000212e+00, -1.643325e-05, -2.384886e-04],
            [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00]],

           [[ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00],
            [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00]]],


          [[[-8.216233e-05,  5.904608e-04,  1.000031e+00, -5.393328e-04],
            [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00]],

           [[ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00],
            [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00]]],


          [[[ 6.103794e-04, -2.515970

In [1]:
# t_tlayer_fwd_gpt2
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_tlayer_gpt2
from model_triton import *
BS, H, N, D = 2, 1, 2, 4
layer_params = init_tlayer_gpt2(D, H, 4*D, 1)
y= torch.randn((BS, N, D), device="cuda")
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
res = t_gpt2_tlayer_fwd(layer_params, y, mask, False)
print(res.shape)
print("---")

from functools import partial
fn = partial(t_gpt2_tlayer_fwd, mask=mask, train=False)
# res = jacrev(fn)(layer_params, y)
# print(len(res), res[0].shape, res[1].shape, res[2].shape, res[3].shape, res[4].shape, res[5].shape)
# print(res[0])
# print(res[1])
# print("---")

# res2 = t_gpt2_tlayer_bkwd_p(layer_params, y, mask, train=False)
# print(len(res2), res2[0].shape, res2[1].shape, res2[2].shape, res2[3].shape, res2[4].shape, res2[5].shape)
# print(res2[0])
# print(res2[1])

res = jacrev(fn, argnums=1)(layer_params, y)
print(res.shape)
print(res)
print("---")

res2 = t_gpt2_tlayer_bkwd_x(layer_params, y, mask, train=False)
print(res2.shape)
print(res2)


torch.Size([2, 2, 4])
---
torch.Size([2, 2, 4, 2, 2, 4])
tensor([[[[[[ 9.999627e-01,  1.201024e-04,  3.038353e-06, -8.580500e-05],
            [ 1.852606e-04,  1.017048e-04, -2.604173e-05, -2.609237e-04]],

           [[ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00],
            [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00]]],


          [[[ 1.891640e-04,  1.000441e+00, -2.049131e-05, -6.100892e-04],
            [-2.495393e-05, -1.998381e-05,  1.972595e-04, -1.523217e-04]],

           [[ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00],
            [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00]]],


          [[[ 6.383702e-05,  1.907873e-04,  9.999929e-01, -2.475114e-04],
            [-1.333327e-04, -8.866435e-05,  4.955884e-04, -2.735914e-04]],

           [[ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00],
            [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00]]],


          [[[ 4.036669e-05, -4.697078

In [44]:
# pos enc test
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_tlayer_gpt2
from model_triton import *
BS, H, N, D = 2, 1, 2, 4
vocab_size = 6
layers = 2
layers_params = init_transformer_gpt2(vocab_size, D, layers, H, 4*D, N)
y= torch.randint(vocab_size, (BS, N), device="cuda")
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
def test_fwd(params, y):
    y = t_embed_fwd(params[0], y)
    return y + params[1][0]
    
res = test_fwd(layers_params[:2], y)
print(res.shape)

print("---")

res = jacrev(test_fwd)(layers_params[:2], y)
print(len(res[1]), res[1][0].shape)
print(res[1][0])
#print_res_shapes(res)

def test_bkwd_p(params, y):
    indices = torch.arange(y.shape[1]).unsqueeze(0).expand(*y.shape)
    y = t_embed_fwd(params[0], y)
    
    
    #print(f'params[1][0].shape', params[1][0].shape)
    #print(f'indices.shape', indices.shape)
    jac = t_embed_bkwd(params[1], indices) 
    jac = (jac[0] / params[1][0].shape[1] * 2, )
    
    return [None, jac]
res2 = test_bkwd_p(layers_params[:2], y)
print(len(res2[1]), res2[1][0].shape)
print(res2[1][0])

torch.Size([2, 2, 4])
---
1 torch.Size([2, 2, 4, 2, 4])
tensor([[[[[1., 0., 0., 0.],
           [0., 0., 0., 0.]],

          [[0., 1., 0., 0.],
           [0., 0., 0., 0.]],

          [[0., 0., 1., 0.],
           [0., 0., 0., 0.]],

          [[0., 0., 0., 1.],
           [0., 0., 0., 0.]]],


         [[[0., 0., 0., 0.],
           [1., 0., 0., 0.]],

          [[0., 0., 0., 0.],
           [0., 1., 0., 0.]],

          [[0., 0., 0., 0.],
           [0., 0., 1., 0.]],

          [[0., 0., 0., 0.],
           [0., 0., 0., 1.]]]],



        [[[[1., 0., 0., 0.],
           [0., 0., 0., 0.]],

          [[0., 1., 0., 0.],
           [0., 0., 0., 0.]],

          [[0., 0., 1., 0.],
           [0., 0., 0., 0.]],

          [[0., 0., 0., 1.],
           [0., 0., 0., 0.]]],


         [[[0., 0., 0., 0.],
           [1., 0., 0., 0.]],

          [[0., 0., 0., 0.],
           [0., 1., 0., 0.]],

          [[0., 0., 0., 0.],
           [0., 0., 1., 0.]],

          [[0., 0., 0., 0.],
       

In [41]:
y = torch.randn((2, N))
torch.arange(y.shape[1]).unsqueeze(0).expand(*y.shape)
#.expand(y.shape[0], 1)
#res

tensor([[0, 1],
        [0, 1]])

In [1]:
# t_gpt2_tlayers_fwd
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_tlayer_gpt2
from model_triton import *
BS, H, N, D = 1, 1, 2, 4
vocab_size = 6
layers = 2
layers_params = init_transformer_gpt2(vocab_size, D, layers, H, 4*D, N)
y= torch.randint(vocab_size, (BS, N), device="cuda")
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
res = t_gpt2_tlayers_fwd(layers_params, y, mask, None, False)
print(res.shape)

print("---")

def print_res_shapes(res):
    print(len(res))
    for it in res:
        print(f'-', len(it))
        for p in it:
            print(f'--', p.shape, p.reshape(-1)[-3:]) # TODO: modify to check different parts

from functools import partial
fn = partial(t_gpt2_tlayers_fwd, mask=mask, indices=None, train=False)
res = jacrev(fn)(layers_params, y)
print_res_shapes(res)
print("---XXXX--")

# def t_gpt2_tlayers_bkwd_p(params, y, mask, indices, train=True): # input: seq_len x
#     indices = torch.arange(y.shape[1], device=y.device).unsqueeze(0).expand(*y.shape) # we ignore indices arg
#     jac_embed = t_embed_bkwd(params[0], y)
#     # Due to tying of embedding and final projection layers,
#     # we need to fill zeroed gradient with respect to biases:
#     jac_embed = [jac_embed[0], torch.zeros(jac_embed[0].shape[:-1], device=y.device)]
#     y = t_embed_fwd(params[0], y)
#     # Reuse t_embed_bkwd to compute jacobian of pos_encoding
#     # Need to account for lack of  1/ sqrt(emb_dim)
#     jac_pos_enc = t_embed_bkwd(params[1], indices) 
#     jac_pos_enc = [jac_pos_enc[0] / params[1][0].shape[1] * 2, ]
#     # TODO XXX: add dropout
#     y = t_dropout(y + params[1][0], train)
    
#     layers_jacs_p = []
#     layers_jacs_x = []
    
#     for layer_params in params[2:-1]:
#         layers_jacs_p.append(t_gpt2_tlayer_bkwd_p(layer_params, y, mask, train))
#         layers_jacs_x.append(t_gpt2_tlayer_bkwd_x(layer_params, y, mask, train))
#         y = t_gpt2_tlayer_fwd(layer_params, y, mask, train)
#     jac_layernorm_p = t_layernorm_bkwd_p(params[-1], y)
#     jac_layernorm_x = t_layernorm_bkwd_x(params[-1], y)    
#     y = t_layernorm_fwd(params[-1], y)
    
#     # Propoagate back
#     def mult_j_in_2d(j_left_2d, j): # we need to do it u
#         j = j.flatten(end_dim=len(y.shape)-1) #TODO XXX: is there a nice way of coding it?
#         j_indim = j.shape[1:]
#         j = j.reshape((y.numel(), -1))
#         res = torch.matmul(j_left_2d, j)
#         return res.reshape(y.shape + j_indim)
#     layers_jacs_x[-1]=torch.einsum('abcdef, defghi -> abcghi', jac_layernorm_x, layers_jacs_x[-1])
#     jac_layernorm_x_2d = jac_layernorm_x.reshape((y.numel(), y.numel()))
#     layers_jacs_p[-1] = [mult_j_in_2d(jac_layernorm_x_2d, j) for j in layers_jacs_p[-1]] 
#     for i in reversed(range(1, len(layers_jacs_p))):
#         layers_jacs_x[i-1]=torch.einsum('abcdef, defghi -> abcghi',layers_jacs_x[i], layers_jacs_x[i-1])
#         jac_layer_2d = layers_jacs_x[i].reshape((y.numel(), y.numel()))
#         layers_jacs_p[i-1] = [mult_j_in_2d(jac_layer_2d, j) for j in layers_jacs_p[i-1]] 
#     jac_pos_enc[0] =torch.einsum('abcdef, defgh -> abcgh', layers_jacs_x[0], jac_pos_enc[0])
#     jac_embed[0] = torch.einsum('abcdef, defgh -> abcgh', layers_jacs_x[0], jac_embed[0])

#     return tuple([jac_embed, jac_pos_enc] + layers_jacs_p + [jac_layernorm_p])

res2 = t_gpt2_tlayers_bkwd_p(layers_params, y, mask, None, train=False)
print_res_shapes(res2) 
#print(res2[0])
#print(res2[1])

# res = jacrev(fn, argnums=1)(layers_params, y)
# print(res.shape)
# print(res)
# print("---")

# res2 = t_tlayer_fwd_gpt2_bkwd_x(layers_params, y, mask, train=False)
# print(res2.shape)
# print(res2)

torch.Size([1, 2, 4])
---
5
- 2
-- torch.Size([1, 2, 4, 6, 4]) tensor([0., 0., 0.], device='cuda:0')
-- torch.Size([1, 2, 4, 6]) tensor([0., 0., 0.], device='cuda:0')
- 1
-- torch.Size([1, 2, 4, 2, 4]) tensor([ 1.819440, -6.423049,  7.003040], device='cuda:0')
- 10
-- torch.Size([1, 2, 4, 4]) tensor([-0.000181,  0.002994,  0.001876], device='cuda:0')
-- torch.Size([1, 2, 4, 4]) tensor([0.000616, 0.010489, 0.002303], device='cuda:0')
-- torch.Size([1, 2, 4, 1, 3, 4, 4]) tensor([-0.028404,  0.028136,  0.080293], device='cuda:0')
-- torch.Size([1, 2, 4, 4, 4]) tensor([ 0.001987, -0.044547,  0.098599], device='cuda:0')
-- torch.Size([1, 2, 4, 4]) tensor([ 0.000751, -0.001716, -0.002677], device='cuda:0')
-- torch.Size([1, 2, 4, 4]) tensor([-0.000567, -0.002559, -0.003033], device='cuda:0')
-- torch.Size([1, 2, 4, 16, 4]) tensor([ 0.147827, -0.076241, -0.100222], device='cuda:0')
-- torch.Size([1, 2, 4, 16]) tensor([ 0.000806,  0.038270, -0.113544], device='cuda:0')
-- torch.Size([1, 2, 4, 

In [15]:
torch.matmul(torch.randn((4,2)), torch.randn(2,5, 3))

RuntimeError: Expected size for first two dimensions of batch2 tensor to be: [2, 2] but got: [2, 5].

In [ ]:
t_tlayer_attn_fwd

In [122]:
# Back to basics
q, k, v = qkv.unbind(dim=2)
q = q.reshape((-1,q.shape[-1]))
k = k.reshape((-1,k.shape[-1]))
v = v.reshape((-1, v.shape[-1]))
print(q, k, v)

def qk_fwd(q, k):
    return torch.matmul(q, k.transpose(-2, -1))

def qkv_fwd(q, k, v):
    return torch.matmul(qk_fwd(q, k), v)

print(qkv_fwd(q, k, v).shape)
print("---")
#res = jacrev(qk_fwd)(q, k)
res = jacrev(qkv_fwd)(q, k, v)
print(res.shape)
print(res)
print("---")

tensor([[-0.144207, -0.994919,  0.033497],
        [-0.169633, -0.113686, -0.300468]], device='cuda:0') tensor([[ 0.432276,  1.592006,  0.251207],
        [ 0.249900, -1.219356, -1.527182]], device='cuda:0') tensor([[-0.585331, -0.783070,  0.105058],
        [ 0.878028, -0.177942,  0.542449]], device='cuda:0')
torch.Size([2, 3])
---
torch.Size([2, 3, 2, 3])
tensor([[[[-0.033605, -2.002479, -1.487947],
          [ 0.000000,  0.000000,  0.000000]],

         [[-0.382970, -1.029678,  0.075037],
          [ 0.000000,  0.000000,  0.000000]],

         [[ 0.180972, -0.494185, -0.802026],
          [ 0.000000,  0.000000,  0.000000]]],


        [[[ 0.000000,  0.000000,  0.000000],
          [-0.033605, -2.002479, -1.487947]],

         [[ 0.000000,  0.000000,  0.000000],
          [-0.382970, -1.029678,  0.075037]],

         [[ 0.000000,  0.000000,  0.000000],
          [ 0.180972, -0.494185, -0.802026]]]], device='cuda:0')
---


In [121]:
def qk_bkwd(q, k):
    return k.transpose
def qkv_bkwd(q, k, v):
    d_qk_wd_dq = jacrev(qk_fwd)(q, k)

    res = torch.matmul(v.transpose(1, 0), d_qk_wd_dq.transpose(1, 0).reshape(q.shape[0], -1))
    res = res.reshape(res.shape[0], -1, q.numel())
    res = res.transpose(1, 0)
    res = res.reshape(q.shape + q.shape)
    
    return res

#res2 = qk_bkwd(q, k)
res2 = qkv_bkwd(q, k, v)
print(res2.shape)
print(res2)

torch.Size([2, 3, 2, 3])
tensor([[[[-0.033605, -2.002479, -1.487947],
          [ 0.000000,  0.000000,  0.000000]],

         [[-0.382970, -1.029678,  0.075037],
          [ 0.000000,  0.000000,  0.000000]],

         [[ 0.180972, -0.494185, -0.802026],
          [ 0.000000,  0.000000,  0.000000]]],


        [[[ 0.000000,  0.000000,  0.000000],
          [-0.033605, -2.002479, -1.487947]],

         [[ 0.000000,  0.000000,  0.000000],
          [-0.382970, -1.029678,  0.075037]],

         [[ 0.000000,  0.000000,  0.000000],
          [ 0.180972, -0.494185, -0.802026]]]], device='cuda:0')


In [39]:
torch.block_diag(torch.ones(2), torch.ones(2))

tensor([[1., 1., 0., 0.],
        [0., 0., 1., 1.]])

In [38]:
def test_fn(a):
    return torch.sum(a, dim=1)

aa = torch.randn((2,2))
print(aa)
print(test_fn(aa))
res = jacrev(test_fn)(aa)
print(res)
print(res.shape)

tensor([[-0.8190, -1.8277],
        [-0.7926,  0.6985]])
tensor([-2.6466, -0.0941])
tensor([[[1., 1.],
         [0., 0.]],

        [[0., 0.],
         [1., 1.]]])
torch.Size([2, 2, 2])


In [24]:
def t_test_fwd(x):
    return 0.044715 * torch.pow(x,3)

def t_test_bkwd(x):
    return 0.044715 * 3 * torch.pow(x,2)

def t_test_fwd(x):
    k = math.sqrt(2/math.pi)
    tanh_term = torch.tanh(k * (x + 0.044715 * torch.pow(x,3)))
    return tanh_term

def t_test_bkwd(x):
    k = math.sqrt(2/math.pi)
    tanh_term = torch.tanh(k * (x + 0.044715 * torch.pow(x,3)))
    tanh_dx = (1 - torch.pow(tanh_term, 2)) * k * ( 1 + 3 * 0.044715 * torch.pow(x,2))
    return tanh_dx


# print(vjp(t_test_fwd, aa)[0])
# print(t_test_bkwd(aa) * aa)

# test_aa = torch.tensor(1.1)
# print(grad(t_test_fwd)(test_aa))
# print(t_test_bkwd(test_aa))

print(jacrev(t_test_fwd)(aa))
print(t_test_bkwd(aa))


tensor([[[[0.7855, 0.0000],
          [0.0000, 0.0000]],

         [[0.0000, 0.5382],
          [0.0000, 0.0000]]],


        [[[0.0000, 0.0000],
          [0.7643, 0.0000]],

         [[0.0000, 0.0000],
          [0.0000, 0.7812]]]])
tensor([[0.7855, 0.5382],
        [0.7643, 0.7812]])
